#PySpark Joins - Common Pitfalls and Practical Insights

##ER Diagram: tpch database

Note:  tpch is a database in samples catalog that's **available in Databricks free edition**

<img src="./ERDiagram_Databricks_Samples_catalog_tpch_db.png" alt="ERDiagram_Databricks_Samples_catalog_tpch_db.png"/>

##Example datasets

###Example data 1 (samples.tpch database tables)

Note: The **samples catalog** and the **tpch schema** are available by default in Databricks free edition.

In [0]:
df_customer = spark.read.table("samples.tpch.customer")
df_orders = spark.read.table("samples.tpch.orders")
df_supplier = spark.read.table("samples.tpch.supplier")

df_nation = spark.read.table("samples.tpch.nation")


###Example data 2 (demo on deduping)

In [0]:
from datetime import datetime

#(i). Invoice dataframe v1: Each set of duplicate rows have the same values across all columns
df_invoice_exact_duplicates = spark.createDataFrame(
    [
        (3301, 29620, 100.00, datetime.strptime("2025-10-01", "%Y-%m-%d"), datetime.strptime("2025-10-01 11:00:00", "%Y-%m-%d %H:%M:%S")),
        (3301, 29620, 100.00, datetime.strptime("2025-10-01", "%Y-%m-%d"), datetime.strptime("2025-10-01 11:00:00", "%Y-%m-%d %H:%M:%S")),
        (3301, 29620, 100.00, datetime.strptime("2025-10-01", "%Y-%m-%d"), datetime.strptime("2025-10-01 11:00:00", "%Y-%m-%d %H:%M:%S")),

        (3302, 29620, 50.00, datetime.strptime("2025-10-02", "%Y-%m-%d"), datetime.strptime("2025-10-02 12:00:00", "%Y-%m-%d %H:%M:%S")),
        (3302, 29620, 50.00, datetime.strptime("2025-10-02", "%Y-%m-%d"), datetime.strptime("2025-10-02 12:00:00", "%Y-%m-%d %H:%M:%S")),

        (3303, 29621, 200.00, datetime.strptime("2025-10-03", "%Y-%m-%d"), datetime.strptime("2025-10-03 15:00:00", "%Y-%m-%d %H:%M:%S")),
        (3303, 29621, 200.00, datetime.strptime("2025-10-03", "%Y-%m-%d"), datetime.strptime("2025-10-03 15:00:00", "%Y-%m-%d %H:%M:%S"))
    ],
    "InvoiceKey: int, suppkey: bigint, InvoiceAmount: double, InvoiceDate: date, LastUpdated: timestamp"
)

#(ii). Invoice dataframe v2: Duplicate on business keys only (i.e. [suppkey + InvoiceKey] in this example)
df_invoice_duplicate_business_key = spark.createDataFrame(
    [
        (3301, 29620, 100.00, datetime.strptime("2025-10-01", "%Y-%m-%d"), datetime.strptime("2025-10-01 11:00:00", "%Y-%m-%d %H:%M:%S")),
        (3301, 29620, 120.00, datetime.strptime("2025-10-01", "%Y-%m-%d"), datetime.strptime("2025-10-01 11:15:00", "%Y-%m-%d %H:%M:%S")),
        (3301, 29620, 150.00, datetime.strptime("2025-10-01", "%Y-%m-%d"), datetime.strptime("2025-10-01 11:30:00", "%Y-%m-%d %H:%M:%S")),

        (3302, 29620, 50.00, datetime.strptime("2025-10-02", "%Y-%m-%d"), datetime.strptime("2025-10-02 12:30:00", "%Y-%m-%d %H:%M:%S")),
        (3302, 29620, 55.00, datetime.strptime("2025-10-02", "%Y-%m-%d"), datetime.strptime("2025-10-02 12:45:00", "%Y-%m-%d %H:%M:%S")),

        (3303, 29621, 200.00, datetime.strptime("2025-10-03", "%Y-%m-%d"), datetime.strptime("2025-10-03 15:10:00", "%Y-%m-%d %H:%M:%S")),
        (3303, 29621, 275.00, datetime.strptime("2025-10-03", "%Y-%m-%d"), datetime.strptime("2025-10-03 15:20:00", "%Y-%m-%d %H:%M:%S"))
    ],
    "InvoiceKey: int, suppkey: bigint, InvoiceAmount: double, InvoiceDate: date, LastUpdated: timestamp"
)


###Example data 3 (demo on data matching case-sensitive)

In [0]:
#1. Employee dataframe
df_employee = spark.createDataFrame(
    [
        ('E1001','Kevin','DPT501'),
        ('E1002','David','DPT502'),
        ('E1003','Ben','Dpt502'),
        ('E1004','Linda','DPT503')
    ],
    "EmpID: string, Name: string, DeptID: string"
)

df_employee.display()

#2. Department dataframe
df_department = spark.createDataFrame(
    [
        ('DPT501','Engineering'),
        ('DPT502','Sales'),
        ('DPT503','Marketing'),
        ('DPT525','IT') 
    ],
    "DeptID: string, Name: string"
)

df_department.display()

###Example data 4 (demo on handling NULLs in joins)

In [0]:
#(i) order dataframe with 1 null (to demonstrate how to handle nulls in the joins)
df_orders_nulls_demo_1Null = spark.createDataFrame(
    (
        (50001, 'C101'),
        (None, 'C101'),
        (50002, 'C102'),
        #(None, 'C103'), #Enabled for scenario 3 only
        (50004, 'C104'),
        (50005, 'C105')
    ),
    ("orderid: int, customerid: string")
)

#(ii) order dataframe with 2 nulls (to demonstrate how to handle nulls in the joins)
df_orders_nulls_demo_2Nulls = spark.createDataFrame(
    (
        (50001, 'C101'),
        (None, 'C101'),
        (50002, 'C102'),
        (None, 'C103'),
        (50004, 'C104'),
        (50005, 'C105')
    ),
    ("orderid: int, customerid: string")
)

#(iii) order_lineitems dataframe
df_lineitems_nulls_demo = spark.createDataFrame(
    (
        (50001, 1, 'Apple',5.0, 2),
        (50001, 2, 'Banana',5.0, 2),
        (None, 3, 'Orange',30.0, 1),
        (50002, 1, 'Carrots',3.0, 1),
        (50002, 2, 'Potato',4.0, 2),
        (None, 1, 'Lemon',25.0, 2),
        (None, 2, 'Milk',50.0, 2),
        (50004, 1, 'Bread',5.0, 2),
        (50005, 1, 'Rice',5.0, 2),
        (50005, 2, 'Shampoo',5.0, 2)
    ),
    ("orderid: int, lineitemnumber: int, productkey: string, price: double, quantity: int")
)


###Example data 5 (demo on many-to-many relationship)

In [0]:
#(i) student df
df_student = spark.createDataFrame(
    (
        (101, 'David'),
        (102, 'Vanessa'),
        (103, 'John')
    ),
    ("id: int, name: string")
)

#(ii) subject df
df_subject = spark.createDataFrame(
    (
        (501, 'Mathematics'),
        (502, 'Biology'),
        (503, 'History'),
        (504, 'English')
    ),
    ("id: int, name: string")
)

#(iii) student-subject df (bridge table)
df_bridge_student_subject = spark.createDataFrame(
    (
        (101,501),
        (101,502),
        (101,504),
        (102,502),
        (102,503),
        (103,501),
        (103,502),
        (103,503),
        (103,504)
    ),
    ("student_id: int, subject_id: int")
)


###Example data 6 (demo on One Problem, Many Solutions)

In [0]:
from decimal import Decimal
from datetime import datetime

# (i) serviceprovider df
df_serviceprovider = spark.createDataFrame(
    [
        (201, 'Service Provider 201'),
        (202, 'Service Provider 202'),
        (203, 'Service Provider 203'),
        (204, 'Service Provider 204')
    ],
    "id: int, name: string"
)

# (ii) invoice df
df_invoice = spark.createDataFrame(
    [
        ('INV201001', 201, Decimal('10000.00'), datetime.strptime("2022-01-01", "%Y-%m-%d"), 'Paid'),
        ('INV201002', 201, Decimal('15000.00'), datetime.strptime("2022-01-02", "%Y-%m-%d"), 'Pending'),
        ('INV202001', 202, Decimal('2500.00'), datetime.strptime("2022-01-01", "%Y-%m-%d"), 'Paid'),
        ('INV203001', 203, Decimal('9000.00'), datetime.strptime("2022-01-01", "%Y-%m-%d"), 'Pending')
    ],
    "invoice_code: string, serviceprovider_id: int, invoice_amount: decimal(18,2), invoice_date: date, invoice_payment_status: string"
)

# (iii) invoice category df
df_invoicecategory = spark.createDataFrame(
    [
        ('Invoice Category: 0-5K',Decimal('00.00'),Decimal('5000.00')),
        ('Invoice Category: 5K-10K',Decimal('5000.01'),Decimal('10000.00')),
        ('Invoice Category: 10K-20K',Decimal('10000.01'),Decimal('20000.00')),
        ('Invoice Category: Over 20K',Decimal('20000.01'),Decimal('999999.00'))
    ],
    "category_description: string, lower_bound: decimal(18,2), upper_bound: decimal(18,2)"
)


###Example data 7 (demo on role-playing dimension)

In [0]:
from datetime import datetime

# (i) DimDate df
df_dimdate = spark.createDataFrame(
    [
        (20251001,datetime.strptime('2025-10-01', "%Y-%m-%d"),4,'Wednesday',10,'October', 4, 2025, False, 'Q4-2025'),
        (20251002,datetime.strptime('2025-10-02', "%Y-%m-%d"),5,'Thursday',10,'October', 4, 2025, False, 'Q4-2025'),
        (20251003,datetime.strptime('2025-10-03', "%Y-%m-%d"),6,'Friday',10,'October', 4, 2025, False, 'Q4-2025'),
        (20251004,datetime.strptime('2025-10-04', "%Y-%m-%d"),7,'Saturday',10,'October', 4, 2025, True, 'Q4-2025'),
        (20251005,datetime.strptime('2025-10-05', "%Y-%m-%d"),1,'Sunday',10,'October', 4, 2025, True, 'Q4-2025'),
        (20251006,datetime.strptime('2025-10-06', "%Y-%m-%d"),2,'Monday',10,'October', 4, 2025, False, 'Q4-2025'),
        (20251007,datetime.strptime('2025-10-07', "%Y-%m-%d"),3,'Tuesday',10,'October', 4, 2025, False, 'Q4-2025'),
        (20251008,datetime.strptime('2025-10-08', "%Y-%m-%d"),4,'Wednesday',10,'October', 4, 2025, False, 'Q4-2025'),
        (20251009,datetime.strptime('2025-10-09', "%Y-%m-%d"),5,'Thursday',10,'October', 4, 2025, False, 'Q4-2025'),
        (20251010,datetime.strptime('2025-10-10', "%Y-%m-%d"),6,'Friday',10,'October', 4, 2025, False, 'Q4-2025')
    ],
    ("DateKey: int, FullDate: Date, DayOfWeek: int, DayName: string, Month: int, MonthName: string, Quarter: int, Year: int, IsHoliday: boolean, FiscalPeriod: string")
)

df_dimdate.display()

#(ii) FactSales df
df_factsales = spark.createDataFrame(
    [
        (1000001, 20251001, 20251002, 20251003, 'PKey001', 'CKey301', 10, 100.00),
        (1000002, 20251004, 20251005, 20251006, 'PKey002', 'CKey302', 5, 10.00),
        (1000003, 20251006, 20251007, 20251008, 'PKey003', 'CKey303', 3, 150.00),
        (1000004, 20251007, 20251008, 20251010, 'PKey004', 'CKey304', 12, 16.00)
    ],
    ("transaction_id: int, OrderDateKey: int, ShipDateKey: int, DeliveryDateKey: int, ProductKey: string, CustomerKey: string, QuantitySold: int, UnitPrice: double")
)

df_factsales.display()


##1.0 Common Pitfalls

###1.1 Tricky LEFT OUTER JOIN

**Scenario**: Say, we want to output every customer record from `samples.tpch.customer` table, along with details of only those order records in `samples.tpch.orders` that have an orderdate >= '1998-08-01'

The tricky bit: The filter in this scenario is on the right dataframe rather than on the left dataframe.

**Solution Discussion**:

One may think of applying the filter on orders data either:
- (Option 1): while joining the two dataframes
- (Option 2): after joining the two dtaframes

Given that this is a left outer join. And, more importantly, the filter is on the right dataframe:
- Option 1 would return CORRECT output.
- Option 2 would return INCORRECT output.

The reason being:
* The option 1 would first filter out orders to get only those orders that have an orderdate >= '1998-08-01'. Then, joins them with customer dataframe.
* Whereas, option 2 first joins customers dataframe with orders dataframe. Then, filters the combined dataframe on orderdate >= '1998-08-01'

The following two cells will demonstrate each of these scenarios:

####Correct result (filtered as part of join condition)

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 1 (samples.tpch database tables)
df_cust_alias = df_customer.alias("df_cust_alias") 
df_ord_alias = df_orders.alias("df_ord_alias")

df_result = (
    df_cust_alias
    .join(df_ord_alias, 
        #join condition:
        [
        (df_cust_alias.c_custkey == df_ord_alias.o_custkey) &
        (df_ord_alias.o_orderdate >= "1998-08-01")
        ]
        ,

        "leftouter"
   )
    .select(
    df_cust_alias.c_custkey.alias("cust_key"),
    df_cust_alias.c_name.alias("cust_name"),
    df_ord_alias.o_orderkey.alias("order_key"),
    df_ord_alias.o_orderdate.alias("order_date"),
    df_ord_alias.o_orderstatus.alias("order_status"),
    df_ord_alias.o_totalprice.alias("order_total_price")
    )
    #.filter(col("order_date") >= "1998-08-01") #In case of LEFT OUTER Join, filtering data over here would result in incorrect results!!!
)

df_result.count() #As a best practice, remove the count() call before promoting code to higher environments.

Note: The customer table in samples.tpch database has only 750,000 records. Whereas, the above output has 750,022 rows, which is correct. Because, one customer may have 0 to many order records. That way, 22 customers have 2 orders each in orders table that satisfied the filter `o_orderdate >= "1998-08-01"`

####Incorrect result (filtered after the join)

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 1 (samples.tpch database tables)
df_cust_alias = df_customer.alias("df_cust_alias") 
df_ord_alias = df_orders.alias("df_ord_alias")

df_result = (
    df_cust_alias
    .join(df_ord_alias, 
        #join condition:
        [
        (df_cust_alias.c_custkey == df_ord_alias.o_custkey) #&
        #(df_ord_alias.o_orderdate >= "1998-08-01")
        ]
        ,

        "leftouter"
   )
    .select(
    df_cust_alias.c_custkey.alias("cust_key"),
    df_cust_alias.c_name.alias("cust_name"),
    df_ord_alias.o_orderkey.alias("order_key"),
    df_ord_alias.o_orderdate.alias("order_date"),
    df_ord_alias.o_orderstatus.alias("order_status"),
    df_ord_alias.o_totalprice.alias("order_total_price")
    )
    .filter(col("order_date") >= "1998-08-01") #In case of LEFT OUTER Join, filtering data over here would result in incorrect results!!!
)

df_result.count() #As a best practice, remove the count() call before promoting code to higher environments.

####Key Takeaways: Tricky LEFT OUTER JOIN

If your scenario is a :
* Left outer join, AND
* the filter is on the RIGHT dataframe rather than on the left one

Then, the filter should be part of the join condition itself rather than applying the filter after the join.

####Hypothetical scenario: What if it were an INNER JOIN instead?

**Scenario**: What if the join scenario in the preceding example was such that we just need an INNER JOIN rather than a LEFT-OUTER-JOIN. In such a case, would it make any difference regardless of whether we applied the filter while joining dataframes (i.e. option 1) or after joining the dataframes (i.e. option 2)?

In an INNER JOIN scenario, either option would return correct results.

However, from performance perspective, which one would perform better? That needs to be explored separately. Mind you, the performance also depends on size of data being processed among many other factors. In case of small datasets, either option may provide satisfactory performance.

###1.2 Deduping data before a join

**Scenario**: Say, you want to join two dataframes. However, one of the dataframes has duplicates records. In such a scenario, how to eliminate duplicates from making it to the output?

**Duplicate data - Background info.:**

Types of duplicate data:
* Exact duplicates: Two or more rows have identical values across all columns
* Duplicates on business key: Two or more rows have matching values on business key(s)

The deduping solution depends on the type of duplicate data.



####Scenario 1: Exact duplicates - pick any one of the duplicates

**Duplicates Scenario**: Exact duplicates, wherein two or more rows have identical values across all columns.

For example, in the *df_invoice_exact_duplicates* as below, the InvoiceKey = 3301 has three duplicate rows with the same values across all columns.

| InvoiceKey | suppkey | InvoiceAmount | InvoiceDate | LastUpdated |
|------------|---------|---------------|-------------|-------------|
| 3301 | 29620 | 100 | 2025-10-01 | 2025-10-01T11:00:00.000+00:00 |
| 3301 | 29620 | 100 | 2025-10-01 | 2025-10-01T11:00:00.000+00:00 |
| 3301 | 29620 | 100 | 2025-10-01 | 2025-10-01T11:00:00.000+00:00 |
| 3302 | 29620 | 50 | 2025-10-02 | 2025-10-02T12:00:00.000+00:00 |
| 3302 | 29620 | 50 | 2025-10-02 | 2025-10-02T12:00:00.000+00:00 |
| 3303 | 29621 | 200 | 2025-10-03 | 2025-10-03T15:00:00.000+00:00 |
| 3303 | 29621 | 200 | 2025-10-03 | 2025-10-03T15:00:00.000+00:00 |

**Solution**: Dedupe the *df_invoice_exact_duplicates* dataframe using `dropDuplicates()` or `distinct()` before joining the two dataframes.

Tip: `drop_duplicates()` is an alias for `dropDuplicates()`

**Limitation**: Entire row would have to be duplicate across every column, to be able to be deduplicated by this approach.

In [0]:
from pyspark.sql.functions import col

#Ref dataset 1: Example data 1 (samples.tpch database tables)
#Ref dataset 2: Example data 2 (demo on deduping)
df_supp_alias = df_supplier.alias("df_supp_alias") 

#Dedupe by entire row: use either dropDuplicates() or distinct()
#df_invoice_alias = df_invoice_exact_duplicates.alias("df_invoice_alias").distinct()
df_invoice_alias = df_invoice_exact_duplicates.alias("df_invoice_alias").dropDuplicates()

df_result = (
    df_supp_alias
    .join(df_invoice_alias, 

        (df_supp_alias.s_suppkey == df_invoice_alias.suppkey),

        "inner"
   )
    .select(
    df_supp_alias.s_suppkey.alias("supplier_key"),
    df_supp_alias.s_name.alias("supplier_name"),
    df_invoice_alias.InvoiceKey.alias("Invoice_key"),
    df_invoice_alias.InvoiceAmount.alias("Invoice_amount"),
    df_invoice_alias.InvoiceDate.alias("Invoice_date"),
    df_invoice_alias.LastUpdated.alias("Last_updated")
    )
).sort(col("supplier_key"), col("Invoice_key"))

df_result.display()

####Scenario 2A: Duplicates on business key - pick any one of the duplicates

**Dedupe Scenario**: Duplicates were found on  business keys (suppkey + InvoiceKey). Pick any one of the duplicate rows regardless of the values in rest of the columns were duplicates or not.

For example, in the *df_invoice_duplicate_business_key* as below, the InvoiceKey = 3301 has three duplicate business keys. Additionally, Invoice Amount and Last Updated column values differed between the duplicate rows.

| InvoiceKey | suppkey | InvoiceAmount | InvoiceDate | LastUpdated |
|------------|---------|---------------|-------------|-------------|
| 3301 | 29620 | 100 | 2025-10-01 | 2025-10-01T11:00:00.000+00:00 |
| 3301 | 29620 | 120 | 2025-10-01 | 2025-10-01T11:15:00.000+00:00 |
| 3301 | 29620 | 150 | 2025-10-01 | 2025-10-01T11:30:00.000+00:00 |
| 3302 | 29620 | 50 | 2025-10-02 | 2025-10-02T12:30:00.000+00:00 |
| 3302 | 29620 | 55 | 2025-10-02 | 2025-10-02T12:45:00.000+00:00 |
| 3303 | 29621 | 200 | 2025-10-03 | 2025-10-03T15:10:00.000+00:00 |
| 3303 | 29621 | 275 | 2025-10-03 | 2025-10-03T15:20:00.000+00:00 |

**Solution**: Use `dropDuplicates([SubsetColList_optional])` by passing it appropriate business keys that you want to dedupe on.

*Tip*: `SubsetColList_optional` param ALWAYS requires a `list[]` to be passed as value, even if just one column name is to be passed as param.

**Limitation**: We cannot guarantee which of those multiple duplicate rows would be picked up as winner. On a small dataset such as in this demo, it was picking up the first row in each set of duplicates as winner. But, it isn't something that we can guarantee. In other words, deduplicated output is non-deterministic.

In [0]:
from pyspark.sql.functions import col

#Ref dataset 1: Example data 1 (samples.tpch database tables)
#Ref dataset 2: Example data 2 (demo on deduping)
df_supp_alias = df_supplier.alias("df_supp_alias") 

#Dedupe using business keys only (i.e. [suppkey + InvoiceKey] in this example)
df_invoice_alias = df_invoice_duplicate_business_key.alias("df_invoice_alias").dropDuplicates(["suppkey","InvoiceKey"])

df_result = (
    df_supp_alias
    .join(df_invoice_alias, 

        (df_supp_alias.s_suppkey == df_invoice_alias.suppkey),

        "inner"
   )
    .select(
    df_supp_alias.s_suppkey.alias("supplier_key"),
    df_supp_alias.s_name.alias("supplier_name"),
    df_invoice_alias.InvoiceKey.alias("Invoice_key"),
    df_invoice_alias.InvoiceAmount.alias("Invoice_amount"),
    df_invoice_alias.InvoiceDate.alias("Invoice_date"),
    df_invoice_alias.LastUpdated.alias("Last_updated")
    )
).sort(col("supplier_key"), col("Invoice_key"))

df_result.display()

####Scenario 2B: Duplicates on business key - dedupe using tie-breaker columns

**Dedupe Scenario**: Duplicates on business keys (suppkey + InvoiceKey) were found. Dedupe them on the business key + tie-breaker column(s).

For example, in the *df_invoice_duplicate_business_key* as below, the InvoiceKey = 3301 has three duplicate business keys. Sort them on the tie-breaker column (LastUpdated) in desc() order and pick the latest of such duplicate rows. 

| InvoiceKey | suppkey | InvoiceAmount | InvoiceDate | LastUpdated |
|------------|---------|---------------|-------------|-------------|
| 3301 | 29620 | 100 | 2025-10-01 | 2025-10-01T11:00:00.000+00:00 |
| 3301 | 29620 | 120 | 2025-10-01 | 2025-10-01T11:15:00.000+00:00 |
| 3301 | 29620 | 150 | 2025-10-01 | 2025-10-01T11:30:00.000+00:00 |
| 3302 | 29620 | 50 | 2025-10-02 | 2025-10-02T12:30:00.000+00:00 |
| 3302 | 29620 | 55 | 2025-10-02 | 2025-10-02T12:45:00.000+00:00 |
| 3303 | 29621 | 200 | 2025-10-03 | 2025-10-03T15:10:00.000+00:00 |
| 3303 | 29621 | 275 | 2025-10-03 | 2025-10-03T15:20:00.000+00:00 |

**Solution approach**: 
* Step 1: Partition data on business key(s)
* Step 2: Sort data asc/desc on tie-breaker columns
* Step 3: Assign row numbers in each partition
* Step 4: Pick first row from within each partition

**Pros**: The deduped output would be deterministic, based on the tie-breaker column that we specify in deduping.

Note: If there were duplicates even on LastUpdated column for a given business key, then the result could be a non-deterministic. In such a case, additional columns may need to be considered to make the output deterministic.

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import Window
from pyspark.sql.functions import row_number

#Ref dataset 1: Example data 1 (samples.tpch database tables)
#Ref dataset 2: Example data 2 (demo on deduping)
df_supp_alias = df_supplier.alias("df_supp_alias") 

#Dedupe using business keys + row_number() on Last_updated column desc.
w = (Window.partitionBy(
    df_invoice_duplicate_business_key.suppkey, df_invoice_duplicate_business_key.InvoiceKey)
    .orderBy(df_invoice_duplicate_business_key.LastUpdated.desc())
)

df_invoice_alias = (df_invoice_duplicate_business_key.alias("df_invoice_alias")
    .withColumn("duplicateRowNo", row_number().over(w))
    .filter(col("duplicateRowNo") == 1)
)

df_result = (
    df_supp_alias
    .join(df_invoice_alias, 

        (df_supp_alias.s_suppkey == df_invoice_alias.suppkey),

        "inner"
   )
    .select(
    df_supp_alias.s_suppkey.alias("supplier_key"),
    df_supp_alias.s_name.alias("supplier_name"),
    df_invoice_alias.InvoiceKey.alias("Invoice_key"),
    df_invoice_alias.InvoiceAmount.alias("Invoice_amount"),
    df_invoice_alias.InvoiceDate.alias("Invoice_date"),
    df_invoice_alias.LastUpdated.alias("Last_updated")
    )
).sort(col("supplier_key"), col("Invoice_key"))

df_result.display()

####Key Takeaways: Deduping scenarios

| Deduplication Scenario | Recommended deduping approach |
|------------------------|-----------------------|
| Scenario 1: Exact duplicates | Pick any one of the duplicate rows by using `dropDuplicate()` |
| Scenario 2A: Duplicates on business key columns. As deduped output, any one of the duplicate rows is acceptable. | Dedupe by using `dropDuplicate(ListOfBizKeys)` by explicitly specifying business key columns to dedupe on. Limitation: No guarantee on which one of the multiple duplicate rows is picked up as deduped row. The result might be **non-deterministic** |
| Scenario 2B: Duplicates on business key columns. (similar to scenario 22A). But, additionally, we want to be able to specify tie-breaker criteria. | Four steps in the solution: Step 1: Partition dataframe on business key columns; Step 2: Sort data on the tie-breaker columns in asc/desc order as required; Step 3: Assign row numbers within each set of duplicate rows; Step 4: Pick the first row in each group of duplicates. Benefit: Finer control on deduping. The output will be **deterministic**. |


###1.3 Data matching is case-sensitive

####Scenario 1: joins on columns of string data type

Say, we want to join two tables - `employee` and `department` DeptID. In department table, the DeptID values have a prefix of DPT (all in upper case). e.g. DPT501. However, in employee table: One of the employee records has a DeptID of 'Dpt502' rather than all upper case DPT502.

In such a case, in the output, would empID: E1003, be able to find a matching department record on 'Dpt502' == 'DPT502'?

**df_employee**
| EmpID | Name | DeptID |
|-------|------|--------|
| E1001 | Kevin | DPT501 |
| E1002 | David | DPT502 |
| E1003 | Ben | Dpt502 (Notice the prefix was Dpt rather than DPT) |
| E1004 | Linda | DPT503 |


**df_department**
| DeptID | Name |
|--------|------|
| DPT501 | Engineering |
| DPT502 | Sales |
| DPT503 | Marketing |
| DPT525 | IT |

**Answer**: No. empID: E1003 wouldn't find a matching department record. Because, the matching on column values is case-sensitive, as demonstrated in the below query. In employee table, for empID: E1003, the DeptID value was 'Dpt502'. Whereas, in Department table the DeptID was 'DPT502' (all upper case).

**df_employee**
| EmpID | Name | DeptID |
|-------|------|--------|
| E1001 | Kevin | DPT501 |
| E1002 | David | DPT502 |
| E1003 | Ben | Dpt502 |
| E1004 | Linda | DPT503 |


**df_department**
| DeptID | Name |
|--------|------|
| DPT501 | Engineering |
| DPT502 | Sales |
| DPT503 | Marketing |
| DPT525 | IT |


In [0]:
from pyspark.sql.functions import col

df_emp_alias = df_employee.alias("df_emp_alias") 
df_dept_alias = df_department.alias("df_dept_alias")

df_result = (
    df_emp_alias
    .join(df_dept_alias, 

        (df_emp_alias.DeptID == df_dept_alias.DeptID),

        "inner"
   )
    .select(
    df_emp_alias.EmpID.alias("Employee_ID"),
    df_emp_alias.Name.alias("Employee_Name"),
    df_emp_alias.DeptID.alias("DeptID_AsInEmpTable"),
    df_dept_alias.DeptID.alias("DeptID_AsInDeptTable"),
    df_dept_alias.Name.alias("Department_Name")
    )
    .sort(col("Employee_ID").asc())
)

df_result.display()

####Scenario 2: Filtering on hard-coded string literals

For example, in the below code, at line# 11/12:
- if the filter value were specified as 'Japan', it would return no rows
- if the filter value were specified as 'JAPAN', it would return data

Because, in the underlying table, the values for this column was all in upper case.

In [0]:
from pyspark.sql.functions import col

#Ref dataset 1: Example data 1 (samples.tpch database tables)
df_nat_alias = df_nation.alias("df_nat_alias")
df_cust_alias = df_customer.alias("df_cust_alias") 

df_result = (
    df_nat_alias
    .join(df_cust_alias, 

        (df_nat_alias.n_nationkey == df_cust_alias.c_nationkey) & 
        #(df_nat_alias.n_name == 'Japan'), #Filter is case-sensitive
        (df_nat_alias.n_name == 'JAPAN'), #Filter is case-sensitive

        "inner"
   )
    .select(
    df_nat_alias.n_nationkey.alias("nation_key"),
    df_nat_alias.n_name.alias("nation_name"),
    df_cust_alias.c_custkey.alias("cust_key"),
    df_cust_alias.c_name.alias("cust_name")
    )
)

df_result.display()

####Key takeaway: Data matching is case-sensitive

The data matching in Spark (pySpark / SQL) is case-sensitive. It effects:
* table joins,
* filters.

So, please be mindful of it.

In [0]:
%sql

select * from samples.tpch.nation
where n_name = 'Japan'

###1.4 Handling NULLs in Joins? Proceed with Caution

####Background Knowledge

The default behaviour of `NULL` / `None` values in data:
* When a `NULL` is compared with any not-null value, it returns `NULL`
* When a `NULL` is compared with another NULL value, it still returns `NULL`

Whereas, when `eqNullSafe()` is used in comparison of column values:
* When a `NULL` is compared with any not-null value, `false` is returned instead of the `NULL` as in default behaviour
* When a `NULL` is compared with another NULL value, `true` is returned instead of the `NULL` as in default behaviour


####Scenario 1: What if I don't handle NULLs in joins?

**Join Scenario**: Say, we have two dataframes to join:
* df_orders_nulls_demo_1Null
* df_lineitems_nulls_demo

As shown in the sample data as below, we want to join both these dataframes on `orderid` column. However, as can be noticed in the data, there is a single NULL value in the `df_orders`. And, three NULL values in the `df_lineitems`.

As a first scenario, let's see what happens if don't do anything special on these NULLs in the key column being used in joining these two dataframes.

**1. df_orders_nulls_demo_1Null**:

| customerid |orderid | 
|------------|--------|
| C101 | 50001 |
| C101 | null |
| C102 | 50002 |
| C104 | 50004 |
| C105 | 50005 |

**2. df_lineitems_nulls_demo**:

| orderid | lineitemnumber | productkey | price | quantity |
|---------|----------------|------------|-------|----------|
| 50001 | 1 | Apple | 5 | 2 |
| 50001 | 2 | Banana | 5 | 2 |
| null | 3 | Orange | 30 | 1 |
| 50002 | 1 | Carrots | 3 | 1 |
| 50002 | 2 | Potato | 4 | 2 |
| null | 1 | Lemon | 25 | 2 |
| null | 2 | Milk | 50 | 2 |
| 50004 | 1 | Bread | 5 | 2 |
| 50005 | 1 | Rice | 5 | 2 |
| 50005 | 2 | Shampoo | 5 | 2 |


In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 3 (demo on handling NULLs in joins)
df_ord_dummy_alias = df_orders_nulls_demo_1Null.alias("df_ord_dummy_alias") 
df_lineitem_dummy_alias = df_lineitems_nulls_demo.alias("df_lineitem_dummy_alias")

df_result = (
    df_ord_dummy_alias
    .join(df_lineitem_dummy_alias, 

        (df_ord_dummy_alias.orderid == df_lineitem_dummy_alias.orderid),

        "inner"
   )
    .select(
    df_ord_dummy_alias.orderid.alias("Order_Id_In_Orders"),
    df_ord_dummy_alias.customerid.alias("Customer_Id"),
    df_lineitem_dummy_alias.orderid.alias("Order_Id_In_LineItem"),
    df_lineitem_dummy_alias.lineitemnumber.alias("LineItemNumber"),
    df_lineitem_dummy_alias.productkey.alias("Product_Key"),
    df_lineitem_dummy_alias.price.alias("Price"),    
    df_lineitem_dummy_alias.quantity.alias("Quantity")
    )
)

df_result.display()

Comments - scenario 1 (default behaviour on NULLs in a join)
* Ignoring rows that have NULLs on the join key, the remaining 7 rows were returned.

####Scenario 2: Single NULL in join Key in one of the two tables

**Join Scenario**: This scenario is very similar to that of scenario 1, with the exception of the variation in the join condition as below:

In scenario 1 - join condition:
* `(df_ord_dummy_alias.orderid == df_lineitem_dummy_alias.orderid)`

In scenario 2 - join condition:
* `(df_ord_dummy_alias.orderid.eqNullSafe(df_lineitem_dummy_alias.orderid))`

As a result of using `eqNullSafe` in scenario 2, the query treats as if NULL == NULL.

Accordingly, the output returns 10 rows.

Please note that in this scenario, in df_orders there was only a single NULL value in the key column (orderId), while in the 2nd df (i.e. df_lineitem) there were 3 NULLs in the key column.


In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 3 (demo on handling NULLs in joins)
df_ord_dummy_alias = df_orders_nulls_demo_1Null.alias("df_ord_dummy_alias") 
df_lineitem_dummy_alias = df_lineitems_nulls_demo.alias("df_lineitem_dummy_alias")

df_result = (
    df_ord_dummy_alias
    .join(df_lineitem_dummy_alias, 

        #(df_ord_dummy_alias.orderid == df_lineitem_dummy_alias.orderid), # scenario 1
        (df_ord_dummy_alias.orderid.eqNullSafe(df_lineitem_dummy_alias.orderid)), # scenario 2

        "inner"
   )
    .select(
    df_ord_dummy_alias.orderid.alias("Order_Id_In_Orders"),
    df_ord_dummy_alias.customerid.alias("Customer_Id"),
    df_lineitem_dummy_alias.orderid.alias("Order_Id_In_LineItem"),
    df_lineitem_dummy_alias.lineitemnumber.alias("LineItemNumber"),
    df_lineitem_dummy_alias.productkey.alias("Product_Key"),
    df_lineitem_dummy_alias.price.alias("Price"),    
    df_lineitem_dummy_alias.quantity.alias("Quantity")
    )
)

df_result.display()

####Scenario 3: Multiple NULLs in join key in both the tables

**Join scenario 3**: Continuing from scenario 2, in this scenario, say we have two NULL values in the key column of df_orders, unlike just a single NULL value as in scenario 2.

Let's see what happens when both of the two dataframes have more than one NULL value in the join column(s).

Note: For this particular scenario, a modified version of the df_order dataframe as below is used:
* `df_orders_nulls_demo_2Nulls`

Notice that when compared to the previous df_orders, this one has an extra rows (where orderId was NULL and customerid = C103)

**df_orders_nulls_demo_2Nulls**:

| customerid | orderid |
|------------|---------|
| C101 | 50001 |
| C101 | null |
| C102 | 50002 |
| C103 (new to scenario 3) | null |
| C104 | 50004 |
| C105 | 50005 |

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 3 (demo on handling NULLs in joins)
df_ord_dummy_alias = df_orders_nulls_demo_2Nulls.alias("df_ord_dummy_alias") 
df_lineitem_dummy_alias = df_lineitems_nulls_demo.alias("df_lineitem_dummy_alias")

df_result = (
    df_ord_dummy_alias
    .join(df_lineitem_dummy_alias, 

        #(df_ord_dummy_alias.orderid == df_lineitem_dummy_alias.orderid),
        (df_ord_dummy_alias.orderid.eqNullSafe(df_lineitem_dummy_alias.orderid)),

        "inner"
   )
    .select(
    df_ord_dummy_alias.orderid.alias("Order_Id_In_Orders"),
    df_ord_dummy_alias.customerid.alias("Customer_Id"),
    df_lineitem_dummy_alias.orderid.alias("Order_Id_In_LineItem"),
    df_lineitem_dummy_alias.lineitemnumber.alias("LineItemNumber"),
    df_lineitem_dummy_alias.productkey.alias("Product_Key"),
    df_lineitem_dummy_alias.price.alias("Price"),    
    df_lineitem_dummy_alias.quantity.alias("Quantity")
    )
)

df_result.sort(col("Customer_id")).display()

####Handling NULLs in Joins? Proceed with Caution

Given that in scenario 3 (Multiple NULLs in join key in both the tables), we have two NULLs in the join key column of 1st df and three NULLs in the join key column of 2nd df, it resulted in a cartisian product of 6 rows (2x3) representing the NULL as key, which isn't correct.

So, if we want to use `eqNullSafe` while joining two dataframes to handle NULLs in key columns, we have to ensure:
* one of the two dataframes being joined cannot have more than 1 NULL value in the key column.
* if both dataframes have more than 1 NULL value in the key column, that would result in INCORRECT output.

Moreover, grouping all the rows with the NULL as key column may not necessarily be correct from business perspective. NULL stands for unknown. That way, grouping multiple UNKNOWNs into one group may or may not necessarily make business sense.

Additionally, the key columns on which we want to join dataframes, are expected to be always containing NOT NULL values. Having said that, if there were lots of NULLs in the key columns, it is most likely a data quality issue. And, it would require an investigation of its own.

For these reasons, proceed with caution if you want to handle NULLs while joining two columns. 


####Key Takeaways: Handling NULLs in joins

* Ideally, DO NOT ALLOW NULLs in key columns. 
* If NULLs are allowed in key-columns, do have data quality checks in place regardless of whether you are handling NULLs in key columns using `eqNullSafe` or using any other approach
* If handling NULLs in key columns, ensure there is no more than 1 NULL value in the key column of one of the two dataframes being joined to avoid incorrect results

###1.5 Handling Many-to-Many relationship using a bridge

**Theory**: 

A couple of basic examples on many-to-many relationship:
* Student-Subjects relationship: One student may study many subjects and a subject can be taken up by many students.
* Book-Author relationship: One author may write many books and a book may be written by many authors.

A typical solution for many-to-many relationship is to have a bridge table between the dataframes that were otherwise in a many-to-many relationship.

**Join scenario**: Say, we have a student table and subject table. Obviously, they were in a many-to-many relationship. So, to resolve the many-to-many relationship into one-to-many relationship, a bridge table was introduced in between the two.

Once a many-to-many relationship is resolved into two sets of one-to-many relationships, writing joins between the three dataframes is no different to any multi-table join.

**df_student dataframe**:

| id | name |
|----|------|
| 101 | David |
| 102 | Sarah |
| 103 | John |

**df_subject dataframe**:
| id | name |
|----|------|
| 501 | Mathematics |
| 502 | Biology |
| 503 | History |
| 504 | English |

**df_bridge_student_subject dataframe**:
| student_id | subject_id |
|------------|------------|
| 101 | 501 |
| 101 | 502 |
| 101 | 504 |
| 102 | 502 |
| 102 | 503 |
| 103 | 501 |
| 103 | 502 |
| 103 | 503 |
| 103 | 504 |

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 4 (demo on many-to-many relationship)
df_student_alias = df_student.alias("df_student_alias") 
df_subject_alias = df_subject.alias("df_subject_alias")
df_bridge_stud_sub_alias = df_bridge_student_subject.alias("df_bridge_stud_sub_alias")

df_result = (
    df_student_alias
    #Join Student to the bridge table
    .join(df_bridge_stud_sub_alias, 

        (df_student_alias.id == df_bridge_stud_sub_alias.student_id),

        "inner"
    )
    #Join Subject to the bridge table
    .join(df_subject_alias, 

        (df_bridge_stud_sub_alias.subject_id == df_subject_alias.id),
    
        "inner"      
    )
    .select(
    df_student_alias.id.alias("student_id"),
    df_student_alias.name.alias("student_name"),
    df_subject_alias.id.alias("subject_id"),
    df_subject_alias.name.alias("subject_name")
    )
    #Tip: Avoid sort() unless required
    .sort(col("student_id").asc(), col("subject_id").asc()) #For code brevity, prefer referencing aliased column names in sort() by using col()

)

df_result.display()


####Key Takeaways: Many-to-Many relationship

If two tables are in a many-to-many relationship, such data models would need to be revised to resolve the many-to-many relationship. One approach is to introduce a bridge table in between them, to convert it into two one-to-many relationship.

Once a bridge table is introduced between two tables that were otherwise in many-to-many relationship, writing joins between them is no different to joining multi-tables.

###1.6 Use Broadcast join with care!


####Theory

Broadcast join is a performance optimization technique, wherein you can pass smaller of the two dataframes being joined, to the `broadcast()` function. The table is broadcast by the driver, who copies it to all worker nodes.

When executing joins, Spark automatically broadcasts tables less than 10MB; however, we may adjust this threshold to broadcast even larger tables. If you’re running a driver with a lot of memory (32GB+), you can safely raise the broadcast thresholds to something like 200MB.

Nevertheless, Databricks recommends to always explicitly broadcast smaller tables using hints or PySpark broadcast function, even if AQE can automatically broadcast smaller tables for us.

Ref: https://www.databricks.com/discover/pages/optimize-data-workloads-guide

So, let's see an example of PySpark `broadcast()` function.

**Scenario**: Join `df_customer` dataframe (contains 750,000 rows) with `df_nation` dataframe (which has only 25 rows). Given the `df_nation` dataframe is very small, let's use `broadcast()` function on the `df_nation` dataframe.

Note: Both `df_customer` and `df_nation` dataframes were populated earlier in this notebook from the `samples.tpch database`.

####Broadcast join example

In [0]:
from pyspark.sql.functions import broadcast, col

#Ref dataset: Example data 1 (samples.tpch database tables)
#df_customer.count() #750,000
#df_nation.count() #25 rows

#Tip: Alternatively, you can call broadcast() function, before using the df in a join 
#df_nat_bc_alias = broadcast(df_nation)

df_result = (
    df_customer.alias("df_cust_alias")
    #Calling broadcast() function on a dataframe marks a DataFrame as small enough for use in broadcast joins.
    .join(broadcast(df_nation).alias("df_nat_bc_alias"),
          
          #Join condition
          #Note: Due to inline-aliasing of dataframes, we cannot use dot notation to access columns.
          [col("df_cust_alias.c_nationkey") == col("df_nat_bc_alias.n_nationkey")],
          "inner"
    )
    .select(
        col("df_cust_alias.c_custkey").alias("customer_key"),
        col("df_cust_alias.c_name").alias("customer_name"), 
        col("df_nat_bc_alias.n_name").alias("nation_name")
    )
)

#df_result.explain(extended=False) #To analyze Broadcast Join
df_result.display()


####Key Takeaway: Use broadcast join with care!

Consider broadcasting a dataframe only when it is small. 10MB is the default size to broadcast tables automatically. However, we may adjust this threshold to broadcast even larger tables. If you’re running a driver with a lot of memory (32GB+), you can safely raise the broadcast thresholds to something like 200MB.

If a dataframe being broadcast is not small as we intended it to be, it could result in Out of Memory (OOM) errors.

As recommended by Databricks, never broadcast a table bigger than 1GB because broadcast happens via the driver and a 1GB+ table could cause OOM on the driver.

So, use broadcast join with care!

####Best practices (Databricks link)

Ref: https://www.databricks.com/discover/pages/optimize-data-workloads-guide

##2.0 Practical Insights

###2.1 One Problem, Many Solutions

**Problem scenario**: Let's say we want to get the list of supplier names that have at least one invoice with a payment_status of 'Paid'. We aren't interested in the invoice details itself in the output.

**df_serviceprovider**:

| id | name |
|----|------|
| 201 | Service Provider 201 |
| 202 | Service Provider 202 |
| 203 | Service Provider 203 |
| 204 | Service Provider 204 |

**df_invoice**:

| invoice_code | serviceprovider_id | invoice_amount | invoice_date | invoice_payment_status |
|--------------|-------------|----------------|--------------|------------------------| 
| INV201001 | 201 | 10000.00 | 2022-01-01 | Paid |
| INV201002 | 201 | 15000.00 | 2022-01-02 | Pending |
| INV202001 | 202 | 7500.00 | 2022-01-01 | Paid |
| INV203001 | 203 | 9000.00 | 2022-01-01 | Pending |

**Solution**:

In this join scenario, there are at least four different ways to write the corresponding PySpark query. 
* Using Left-semi join
* Using `isin()`
* Using Inner join
* Using Left outer join by filtering out unmatched rows


####Using Left-SEMI join

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 5 (demo on One Problem, Many Solutions)
df_sp = df_serviceprovider.alias("df_sp")
df_inv = df_invoice.alias("df_inv")

df_result = (
    df_sp.join(
        df_inv.filter(col("invoice_payment_status") == "Paid"),
        df_sp.id == df_inv.serviceprovider_id,
        "left_semi"
    )
)

df_result.display()

####Using ISIN()

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 5 (demo on One Problem, Many Solutions)
df_sp = df_serviceprovider.alias("df_sp")
df_inv = df_invoice.alias("df_inv")

# Get a list of qualified supplier Ids first
sp_ids = [row.serviceprovider_id for row in df_inv
    .filter(col("invoice_payment_status") == "Paid")
    .select("serviceprovider_id")
    .distinct()
    .collect() # collect() could impact performance if the list is very long
]

df_result = df_sp.filter(col("id").isin(sp_ids))

display(df_result)

####Using INNER JOIN

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 5 (demo on One Problem, Many Solutions)
df_sp = df_serviceprovider.alias("df_sp")
df_inv = df_invoice.alias("df_inv")

df_result = (
    df_sp.join(
        df_inv.filter(col("invoice_payment_status") == "Paid"),
        df_sp.id == df_inv.serviceprovider_id,
        "inner"
    )
)

df_result.display()

####Using LEFT OUTER JOIN + filter out unmatched rows

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 5 (demo on One Problem, Many Solutions)
df_sp = df_serviceprovider.alias("df_sp")
df_inv = df_invoice.alias("df_inv")

df_result = (
    df_sp.join(
        df_inv.filter(col("invoice_payment_status") == "Paid"),
        df_sp.id == df_inv.serviceprovider_id,
        "left_outer"
    )
    .filter(col("serviceprovider_id").isNotNull())
)

df_result.display()

####Key Takeaways: One Problem, Many Solutions

We may be able to write a query in several different ways. Each approach may produce the same result, but the best choice depends on factors like performance, readability, and personal preference.

So, I would start writing the query in an approach of my personal preference (in this example, left-semi join), and depending on how it performs, I would explore other options.

Note: A given query performs depends on various factors including size of data etc.

###2.2 Joining tables on a range of values

**Join scenario**: Using the invoice_amount in the df_invoice table as below, join it on a range of values (lower_bound / upper_bound) in df_invoicecategory table, to determine the invoice category for each of the invoices.

**df_invoice**:

| invoice_code | supplier_id | invoice_amount | invoice_date | invoice_payment_status |
|--------------|-------------|----------------|--------------|------------------------| 
| INV201001 | 201 | 10000.00 | 2022-01-01 | Paid |
| INV201002 | 201 | 15000.00 | 2022-01-02 | Pending |
| INV202001 | 202 | 7500.00 | 2022-01-01 | Paid |
| INV203001 | 203 | 9000.00 | 2022-01-01 | Pending |

**df_invoicecategory**:

| category_description | lower_bound | upper_bound |
|----------------------|-------------|-------------|
| Invoice Category: 0-5K | 0.00 | 5000.00 |
| Invoice Category: 5K-10K | 5000.01 | 10000.00 |
| Invoice Category: 10K-20K | 10000.01 | 20000.00 |
| Invoice Category: Over 20K | 20000.01 | 999999.00 |

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 5 (demo on One Problem, Many Solutions)
df_inv_alias = df_invoice.alias("df_inv_alias") 
df_invcat_alias = df_invoicecategory.alias("df_invcat_alias")

df_result = (
    df_inv_alias
    .join(df_invcat_alias, 

        #Joining on a range of values
        (df_inv_alias.invoice_amount.between(df_invcat_alias.lower_bound,df_invcat_alias.upper_bound)),

        "leftouter"
   )
    .select(
    df_inv_alias.invoice_code.alias("invoice_code"),
    df_inv_alias.supplier_id.alias("supplier_id"),
    df_inv_alias.invoice_amount.alias("invoice_amount"),
    df_invcat_alias.category_description.alias("invoice_category")
    )
)

df_result.display()

###2.3 Dealing with role playing dimensions

**What's a role playing dimension?**

Say, we have sales transaction records that have multiple key dates associated with it: Order Date, Ship Date, and Delivery Date.

A fact table `FactSales` designed on the sales transaction data would have the below foreign keys to represent the three dates that we were talking about, apart from all other necessary fields:
* OrderDateKey
* ShipDateKey
* DeliveryDateKey

These three foreign keys reference the `DimDate` dimension, which is physically one dimension table.

In other words, the fact table has multiple foreign keys that all point to the same dimension table.

For this reason, dimension tables such as `DimDate` are called as Role-playing dimensions.

**Scenario**: Say, we have:
* Fact Table: `FactSales` (with foreign keys OrderDateKey, ShipDateKey, DeliveryDateKey).
* Dimension Table: `DimDate` (with attributes like date, month, year, day of week).

Role-Playing: The `DimDate` table is linked to the `FactSales` table three times, once for each date key, creating three independent views of the same date dimension.
* DimDate as Order Date (linked to OrderDateKey)
* DimDate as Ship Date (linked to ShipDateKey)
* DimDate as Delivery Date (linked to DeliveryDateKey) 

For each of the transactions in the FactSales, retrieve transaction details as well as the Date and DateName for each of the OrderDates, ShipDates and DeliveryDates

**df_factsales**:

| transaction_id | OrderDateKey | ShipDateKey | DeliveryDateKey | ProductKey | CustomerKey | QuantitySold | UnitPrice |
|----------------|--------------|-------------|-----------------|------------|-------------|--------------|-----------|
| 1000001 | 20251001 | 20251002 | 20251003 | PKey001 | CKey301 | 10 | 100 |
| 1000002 | 20251004 | 20251005 | 20251006 | PKey002 | CKey302 | 5 | 10 |
| 1000003 | 20251006 | 20251007 | 20251008 | PKey003 | CKey303 | 3 | 150 |
| 1000004 | 20251007 | 20251008 | 20251010 | PKey004 | CKey304 | 12 | 16 |

**df_dimdate**:

| DateKey | FullDate | DayOfWeek | DayName | Month | MonthName | Quarter | Year | IsHoliday | FiscalPeriod |
|---------|----------|-----------|---------|-------|-----------|---------|------|-----------|--------------|
| 20251001 | 2025-10-01 | 4 | Wednesday | 10 | October | 4 | 2025 | false | Q4-2025 |
| 20251002 | 2025-10-02 | 5 | Thursday | 10 | October | 4 | 2025 | false | Q4-2025 |
| 20251003 | 2025-10-03 | 6 | Friday | 10 | October | 4 | 2025 | false | Q4-2025 |
| 20251004 | 2025-10-04 | 7 | Saturday | 10 | October | 4 | 2025 | true | Q4-2025 |
| 20251005 | 2025-10-05 | 1 | Sunday | 10 | October | 4 | 2025 | true | Q4-2025 |
| 20251006 | 2025-10-06 | 2 | Monday | 10 | October | 4 | 2025 | false | Q4-2025 |
| 20251007 | 2025-10-07 | 3 | Tuesday | 10 | October | 4 | 2025 | false | Q4-2025 |
| 20251008 | 2025-10-08 | 4 | Wednesday | 10 | October | 4 | 2025 | false | Q4-2025 |
| 20251009 | 2025-10-09 | 5 | Thursday | 10 | October | 4 | 2025 | false | Q4-2025 |
| 20251010 | 2025-10-10 | 6 | Friday | 10 | October | 4 | 2025 | false | Q4-2025 |


**Solution outline**:

Join the `FactSales` table to `DimDate` as three instances:
* 1st instance of `DimDate` representing OrderDates
* 2nd instance of `DimDate` representing ShipDates
* 3rd instance of `DimDate` representing DeliveryDates

Join these four dataframes, very similar to how you would do joins in a multi-table join.

In [0]:
from pyspark.sql.functions import col

#Ref dataset: Example data 6 (demo on role-playing dimension)

df_factsales_alias = df_factsales.alias("df_factsales_alias") 
#Notice how the same df_dimdate is being instantiated into three separate instances: 
# df_orderdate_alias, df_shipdate_alias, df_deliverydate_alias
df_orderdate_alias = df_dimdate.alias("df_orderdate_alias")
df_shipdate_alias = df_dimdate.alias("df_shipdate_alias")
df_deliverydate_alias = df_dimdate.alias("df_deliverydate_alias")

df_result = (
    df_factsales_alias
    #Join factsales to df_orderdate_alias
    .join(df_orderdate_alias, 

        (df_factsales_alias.OrderDateKey == df_orderdate_alias.DateKey),

        "inner"
    )
    #Join factsales to df_shipdate_alias
    .join(df_shipdate_alias,
          (df_factsales_alias.ShipDateKey == df_shipdate_alias.DateKey),
        "inner"      
    )
    #Join factsales to df_deliverydate_alias
    .join(df_deliverydate_alias,
          (df_factsales_alias.DeliveryDateKey == df_deliverydate_alias.DateKey),
        "inner"      
    )
    .select(
        df_factsales_alias.transaction_id.alias("transaction_id"),
        df_orderdate_alias.FullDate.alias("OrderDate"),
        df_orderdate_alias.DayName.alias("OrderDay"),
        df_shipdate_alias.FullDate.alias("ShipDate"),
        df_shipdate_alias.DayName.alias("ShipDay"),
        df_deliverydate_alias.FullDate.alias("DeliveryDate"),
        df_deliverydate_alias.DayName.alias("DeliveryDay"),
        df_factsales_alias.ProductKey.alias("ProductKey"),
        df_factsales_alias.CustomerKey.alias("CustomerKey"),
        df_factsales_alias.QuantitySold.alias("QuantitySold"),
        df_factsales_alias.UnitPrice.alias("UnitPrice")
    )
    #Tip: Avoid sort() unless required
    .sort(col("transaction_id").asc()) #For code brevity, prefer referencing aliased column names in sort() by using col()

)

df_result.display()

